In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import scarplet as sl
from scarplet.WindowedTemplate import Channel

In [ ]:
params = {'scale': 10,
          'age': 10,
          'ang_min': -np.pi / 2,
          'ang_max': np.pi / 2
         }

data = sl.load('data/channelnetwork.tif')
res = sl.match(data, Channel, **params)

In [ ]:
sl.plot_results(data, res)